In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import numpy as np
import PIL.Image
from matplotlib import pylab as P
import saliency.core as saliency

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
training_set = train_datagen.flow_from_directory('../../Covid19-dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 251 images belonging to 3 classes.


In [5]:
test_set = test_datagen.flow_from_directory('../../Covid19-dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [6]:
model = load_model('../../models/covid/covid_vgg1.h5')

In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
print(model.summary())

In [7]:
def call_model_function(images, call_model_args=None, expected_keys=None):
    target_class_idx =  call_model_args[class_idx_str]
    images = tf.convert_to_tensor(images)
    with tf.GradientTape() as tape:
        output_layer = model(images)
        output_layer = output_layer[:, target_class_idx]
        gradients = np.array(tape.gradient(output_layer, images))
        return {saliency.base.INPUT_OUTPUT_GRADIENTS: gradients}

In [13]:
def load_image(file_path):
    im = PIL.Image.open(file_path)
    im = im.resize((224, 224))
    im = np.asarray(im, dtype=np.float32)
    im = np.expand_dims(im, axis=0)
    return im.copy()


In [15]:
# Load and preprocess your image
im_orig = load_image('../../Covid19-dataset/test/Covid/0100.jpeg')
im = test_datagen.standardize(im_orig)

# Make predictions using your model
predictions = model(im)
prediction_class = np.argmax(predictions[0])
class_idx_str = 'class_idx_str'
call_model_args = {class_idx_str: prediction_class}

# Construct the saliency object
integrated_gradients = saliency.IntegratedGradients()

# Baseline is a black image
baseline = np.zeros(im.shape)

# Compute the SmoothGrad Integrated Gradients mask
smoothgrad_integrated_gradients_mask_3d = integrated_gradients.GetSmoothedMask(
  im, call_model_function, call_model_args, x_steps=25, x_baseline=baseline, batch_size=20)

# Visualize the saliency mask
smoothgrad_mask_grayscale = saliency.VisualizeImageGrayscale(smoothgrad_integrated_gradients_mask_3d)

# Render the saliency mask
ShowGrayscaleImage(smoothgrad_mask_grayscale, title='Smoothgrad Integrated Gradients')

FileNotFoundError: [Errno 2] No such file or directory: './your_image.png'